# Labelme to Yolo - obj det

In [8]:
import json
import os
import cv2
from tqdm import tqdm

def labelme_to_yolo(json_path, class_map, image_folder, file_name):
    with open(json_path, 'r') as f:
        data = json.load(f)

    # image_width = data['imageWidth']
    # image_height = data['imageHeight']

    image = cv2.imread(image_folder + file_name+ '.jpg')
    image_height, image_width, channels = image.shape

    yolo_lines = []
    for shape in data['shapes']:
        
        label = shape['label']
        class_index = class_map.get(label, -1)
        if class_index == -1:
            print("Class map out of dict, Please edit class map")
            continue
            
        points = shape['points']
        x_min = min(points, key=lambda x: x[0])[0]
        x_max = max(points, key=lambda x: x[0])[0]
        y_min = min(points, key=lambda x: x[1])[1]
        y_max = max(points, key=lambda x: x[1])[1]


        x_center = (x_min + x_max) / (2.0 * image_width)
        y_center = (y_min + y_max) / (2.0 * image_height)
        width = (x_max - x_min) / image_width
        height = (y_max - y_min) / image_height

        yolo_lines.append(f"{class_index} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")

    return yolo_lines

# Example usage:
if __name__ == '__main__':
    focus_site = "073"
    # focus_folder = './batch1-2/person-phone/crop_phone'
    focus_folder = f'../assets/crop-img/{focus_site}/images/'
    # focus_folder = './person-phone/post-prd-12-2023/post-prd-batch-12-2023/images/'
    # focus_folder = './img/'
    file_list = os.listdir(focus_folder)
    json_files = [filename for filename in file_list if filename.endswith('.json')]


    for json_file in tqdm(json_files):
        # json_file = 'BKF-BLUE-1_phone_ (150).json'
        source_dir = os.path.join(focus_folder, json_file)
        # output_dir = './yolo_label'
        output_dir = f"../assets/crop-img/{focus_site}/labels_coco/"
        class_map = {
            # 'car' : 0
            'person': 0,
            'phone': 67,
            # Add more class mappings as needed
        }

        name = json_file.split('.json')[0]
        os.makedirs(output_dir, exist_ok=True)

        yolo_lines = labelme_to_yolo(source_dir, class_map, image_folder= focus_folder, file_name=name)
        image_name = os.path.splitext(os.path.basename(json_file))[0]
        output_file = os.path.join(output_dir, f"{image_name}.txt")

        with open(output_file, 'w') as f:
            f.write('\n'.join(yolo_lines))

  0%|          | 0/102 [00:00<?, ?it/s]


AttributeError: 'NoneType' object has no attribute 'shape'

In [1]:
# file_list = os.listdir('batch1-2')

# json_files = [filename for filename in file_list if filename.endswith('.json')]

# print(json_files)

In [4]:
import json
  
# Opening JSON file
f = open('BKF-BLUE-1_phone_ (150).json')
  
# returns JSON object as 
# a dictionary
data = json.load(f)
  
# Iterating through the json
# list
for i in data['shapes']:
    print(i)
  
# Closing file
f.close()

{'label': 'person_with_phone', 'points': [[1608.108108108108, 1105.4054054054054], [2132.4324324324325, 1505.4054054054054]], 'group_id': None, 'description': '', 'shape_type': 'rectangle', 'flags': {}}
{'label': 'person', 'points': [[2278.3783783783783, 1086.4864864864865], [2513.5135135135133, 1232.4324324324325]], 'group_id': None, 'description': '', 'shape_type': 'rectangle', 'flags': {}}


In [8]:
import json
import os
def labelme_to_yolo(labelme_annotation, image_width, image_height, class_map=None):
    label = labelme_annotation['label']
    points = labelme_annotation['points']

    x_min = min(points, key=lambda x: x[0])[0]
    x_max = max(points, key=lambda x: x[0])[0]
    y_min = min(points, key=lambda x: x[1])[1]
    y_max = max(points, key=lambda x: x[1])[1]

    x_center = (x_min + x_max) / (2.0 * image_width)
    y_center = (y_min + y_max) / (2.0 * image_height)
    width = (x_max - x_min) / image_width
    height = (y_max - y_min) / image_height

    yolo_format = f"{class_map[label] if class_map else label} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}"
    return yolo_format

In [9]:
# Sample labelme annotation data
labelme_annotation = {
    'label': 'person_with_phone',
    'points': [[1608.108108108108, 1105.4054054054054], [2132.4324324324325, 1505.4054054054054]],
}

# Image width and height (you need to set these values according to your image size)
image_width = 2600
image_height = 2600

# Class index mapping (optional)
class_map = {
    'person': 0,
    'person_with_phone': 1,
    # Add more class mappings as needed
}

yolo_annotation = labelme_to_yolo(labelme_annotation, image_width, image_height, class_map)
print(yolo_annotation)

1 0.719335 0.502079 0.201663 0.153846


# Labelme to Yolo - polygon

In [28]:
import json
import os
import cv2
from tqdm import tqdm

focus_site = "../../../datasets/ID_card/assets/only_license 01-06_2024/datasets onlylicense 01-06_2024/all/all/set 1/images"
os.listdir(focus_site)

['driver-IMG_CPFGS_LG0_CPFGS000001124_LicenseImage_20240525010942.jpeg',
 'driver-IMG_CPFGS_LG0_CPFGS000001124_LicenseImage_20240525010942.json',
 'driver-IMG_CPFGS_LG0_CPFGS000001124_LicenseImage_20240526134755.jpeg',
 'driver-IMG_CPFGS_LG0_CPFGS000001124_LicenseImage_20240526134755.json',
 'driver-IMG_CPFGS_LG0_CPFGS000001124_LicenseImage_20240527150436.jpeg',
 'driver-IMG_CPFGS_LG0_CPFGS000001124_LicenseImage_20240527150436.json',
 'driver-IMG_CPFGS_LG0_CPFGS000003551_LicenseImage_20240205104914.jpeg',
 'driver-IMG_CPFGS_LG0_CPFGS000003551_LicenseImage_20240205104914.json',
 'driver-IMG_CPFGS_LG0_CPFGS000007509_LicenseImage_20240319004720.jpeg',
 'driver-IMG_CPFGS_LG0_CPFGS000007509_LicenseImage_20240319004720.json',
 'driver-IMG_CPFGS_LG0_CPFGS09419_LicenseImage_20240220184345.jpeg',
 'driver-IMG_CPFGS_LG0_CPFGS09419_LicenseImage_20240220184345.json',
 'driver-IMG_CPFGS_LG0_CPFGS10044_LicenseImage_20240305114437.jpeg',
 'driver-IMG_CPFGS_LG0_CPFGS10044_LicenseImage_20240305114437.j

In [29]:
def labelme_to_yolo_instance_segmentation(img_path, class_map, image_folder, file_name):
    with open(image_folder + file_name + '.json', 'r') as file:
        data = json.load(file)

    image = cv2.imread(img_path)
    img_height, img_width, _ = image.shape


    yolo_annotations = []
    
    for shape in data['shapes']:

        label = shape['label']
        points = shape['points']  # List of [x, y] points
        
        # Normalize polygon points
        normalized_polygon = []
        for x, y in points:
            normalized_polygon.append(x / img_width)
            normalized_polygon.append(y / img_height)
        
        # Create YOLO instance segmentation annotation
        class_id = class_map[label]  # Change based on your class mapping
        yolo_annotation = [class_id]
        yolo_annotation.extend(normalized_polygon)
        
        yolo_annotations.append(yolo_annotation)
    
    return yolo_annotations

# Example usage:
image_extensions = ('.jpg', '.jpeg', '.png', '.gif', '.bmp', '.tiff', '.webp')
if __name__ == '__main__':
    focus_site = "set 1"
    focus_folder = f"../../../datasets/ID_card/assets/only_license 01-06_2024/datasets onlylicense 01-06_2024/all/all/{focus_site}/images/"
    output_dir = f"../../../datasets/ID_card/assets/only_license 01-06_2024/datasets onlylicense 01-06_2024/all/all/{focus_site}/labels/"
    file_list = os.listdir(focus_folder)
    img_files = [filename for filename in file_list if filename.endswith(image_extensions)]


    for img_file in tqdm(img_files):
        try:
            source_dir = os.path.join(focus_folder, img_file)
            class_map = {
                'good': 0,
                'bad': 1,
                # Add more class mappings as needed
            }

            name = img_file.split('.')[0]
            

            os.makedirs(output_dir, exist_ok=True)

            yolo_lines = labelme_to_yolo_instance_segmentation(source_dir, class_map, image_folder=focus_folder, file_name=name)
            output_file = os.path.join(output_dir, f"{name}.txt")
            # print(yolo_lines)

            with open(output_file, 'w') as f:
                # Loop through each sublist
                for line in yolo_lines:
                    # Convert the sublist to a space-separated string and write to the file
                    f.write(' '.join(map(str, line)) + '\n')
        except:
            print(img_file)

100%|██████████| 969/969 [00:30<00:00, 32.17it/s]


# Convert yolo to labelme - obj det

In [1]:
import os
import json
import cv2
from tqdm import tqdm

label_files = os.listdir('./person-phone/post-prd-12-2023/post-prd-batch-12-2023/phone/labels/')
label_files[:5]

['BKF-Inspection_phone_2023-12-06_09-37-07.436000.txt',
 'BKF-Inspection_phone_2023-12-28_08-57-11.100000.txt',
 'BKF-Inspection_phone_2023-12-28_08-19-22.174000.txt',
 'BKF-Inspection_phone_2023-12-28_08-02-17.500000.txt',
 'BKF-Inspection_phone_2023-12-28_07-58-23.063000.txt']

In [2]:
class_map = {
            0.0: 'person',
            1.0: 'phone',
            # Add more class mappings as needed
        }

for file_txt_name in tqdm(label_files):
    file_name = file_txt_name.split('.txt')[0]
    image_folder = './person-phone/post-prd-12-2023/post-prd-batch-12-2023/phone/images/'
    txt_folder = './person-phone/post-prd-12-2023/post-prd-batch-12-2023/phone/labels/'

    image = cv2.imread(image_folder + file_name+ '.jpg')
    image_height, image_width, channels = image.shape

    
    yolo_labels = open(txt_folder + file_name + '.txt', "r").readlines()

    labelme_annotations = []
    # For each YOLO label
    for yolo_label in yolo_labels:
        values = yolo_label.split()
        class_id, x_center, y_center, width, height = map(float, values)
        # Calculate LabelMe coordinates
        x_top_left = int((x_center - width / 2) * image_width)
        y_top_left = int((y_center - height / 2) * image_height)
        x_bottom_right = int((x_center + width / 2) * image_width)
        y_bottom_right = int((y_center + height / 2) * image_height)
        
        labelme_annotation = {
            "label": class_map[class_id],  # Replace with actual class name
            "points": [[x_top_left, y_top_left], [x_bottom_right, y_bottom_right]],
            "description": "",
            "shape_type": "rectangle"
        }
        
        # Create or append to the LabelMe JSON
        # You'll need to manage annotations per image in your actual code
        labelme_annotations.append(labelme_annotation)

    json_file_full = {
        "version": "5.3.0a0",
        "flags": {},
        "shapes" : labelme_annotations,
        "imagePath": file_name + '.jpg',
        "imageData" : None
    }


    with open("./img/" + file_name + ".json", "w") as json_file:
        json.dump(json_file_full, json_file)


  0%|          | 0/1342 [00:00<?, ?it/s]

100%|██████████| 1342/1342 [00:22<00:00, 58.55it/s]


# Convert yolo to labelme - polygon

In [18]:
import os
import json
import cv2
from tqdm import tqdm

IMG_FOLDER = "../../../datasets/ID_card/assets/only_license 01-06_2024/datasets onlylicense 01-06_2024/set1/all/train/images"
LABEL_FOLDER = "../../../datasets/ID_card/assets/only_license 01-06_2024/datasets onlylicense 01-06_2024/set1/all/train/labels"
OUT_PATH = "../../../datasets/ID_card/assets/only_license 01-06_2024/datasets onlylicense 01-06_2024/set1/all/train/json"

In [25]:
image_paths = []
image_name_arr = []

for root, dirs, files in os.walk(IMG_FOLDER):
    for file in files:
        
        if file.lower().endswith((".png", ".jpg", ".jpeg", ".gif", ".bmp", ".webp", ".avif")):
            image_path = os.path.join(root, file)
            image_name_arr.append(file)
            image_paths.append(image_path)

label_paths = []
label_name_arr = []

for root, dirs, files in os.walk(LABEL_FOLDER):
    for file in files:
        if file.lower().endswith((".txt")):
            label_path = os.path.join(root, file)
            label_name_arr.append(file)
            label_paths.append(label_path)

print(image_paths)
print(image_name_arr)
# print(files_name)
print(label_name_arr)

['../../../datasets/ID_card/assets/only_license 01-06_2024/datasets onlylicense 01-06_2024/set1/all/train/images\\driver-IMG_CPFGS_LG0_CPFGS400084833_LicenseImage_20240202221431.jpeg', '../../../datasets/ID_card/assets/only_license 01-06_2024/datasets onlylicense 01-06_2024/set1/all/train/images\\driver-IMG_CPFGS_LG0_CPFGS000001124_LicenseImage_20240525010942.jpeg', '../../../datasets/ID_card/assets/only_license 01-06_2024/datasets onlylicense 01-06_2024/set1/all/train/images\\driver-IMG_CPFGS_LG0_CPFGS000001124_LicenseImage_20240526134755.jpeg', '../../../datasets/ID_card/assets/only_license 01-06_2024/datasets onlylicense 01-06_2024/set1/all/train/images\\driver-IMG_CPFGS_LG0_CPFGS000001124_LicenseImage_20240527150436.jpeg', '../../../datasets/ID_card/assets/only_license 01-06_2024/datasets onlylicense 01-06_2024/set1/all/train/images\\driver-IMG_CPFGS_LG0_CPFGS000003551_LicenseImage_20240205104914.jpeg', '../../../datasets/ID_card/assets/only_license 01-06_2024/datasets onlylicense 

In [26]:
import json
import numpy as np



def yolo_to_labelme(image_path, image_name, txt_path, output_json_path, class_names=None):
    # Load the image to get its dimensions
    import cv2
    image = cv2.imread(image_path)
    height, width, _ = image.shape

    # Initialize the LabelMe JSON structure
    labelme_data = {
        "version": "5.3.0a0",  # LabelMe version (can adjust based on your tool version)
        "flags": {},
        "shapes": [],
        "imagePath": image_name,
        "imageData": None,
        "imageHeight": height,
        "imageWidth": width
    }

    # Read the YOLO instance segmentation data
    with open(txt_path, 'r') as file:
        lines = file.readlines()

    for line in lines:
        data = line.strip().split()
        class_id = int(data[0])  # First value is the class ID
        points = np.array(data[1:], dtype=np.float32).reshape(-1, 2)  # Remaining values are x, y pairs
        
        # Denormalize points to image dimensions
        points[:, 0] *= width   # x-coordinate
        points[:, 1] *= height  # y-coordinate
        
        # Convert points to list of lists for JSON compatibility
        points_list = points.tolist()

        # Determine label
        label = class_names[class_id]

        # Create a shape entry for LabelMe
        shape = {
            "label": label,
            "points": points_list,
            "group_id": None,
            "shape_type": "polygon",
            "flags": {}
        }

        labelme_data["shapes"].append(shape)

    # Save the JSON to file
    with open(output_json_path + ".json", 'w') as json_file:
        json.dump(labelme_data, json_file, indent=4)




In [27]:
from tqdm import tqdm
# Optional: Provide class names for labels
class_names = {
    0 : "good",
    1 : "bad"
}

for image_path, image_name, label_name, txt_path in tqdm(zip(image_paths[:], image_name_arr[:], label_name_arr[:], label_paths[:])):

    file_name = label_name.split('.txt')[0]

    output_json_path = os.path.join(OUT_PATH, file_name)
    yolo_to_labelme(image_path, image_name, txt_path, output_json_path, class_names)

In [36]:
# import base64


# image_width = 2600
# image_height = 2600


# class_map = {
#             0.0: 'person',
#             1.0: 'phone',
#             # Add more class mappings as needed
#         }
# labelme_annotations = []

# # For each YOLO label
# for yolo_label in yolo_labels:
#     values = yolo_label.split()
#     class_id, x_center, y_center, width, height = map(float, values)
#     # Calculate LabelMe coordinates
#     x_top_left = int((x_center - width / 2) * image_width)
#     y_top_left = int((y_center - height / 2) * image_height)
#     x_bottom_right = int((x_center + width / 2) * image_width)
#     y_bottom_right = int((y_center + height / 2) * image_height)
    
#     labelme_annotation = {
#         "label": class_map[class_id],  # Replace with actual class name
#         "points": [[x_top_left, y_top_left], [x_bottom_right, y_bottom_right]],
#         "description": "",
#         "shape_type": "rectangle"
#     }
    
#     # Create or append to the LabelMe JSON
#     # You'll need to manage annotations per image in your actual code
#     labelme_annotations.append(labelme_annotation)

# json_file_full = {
#     "version": "5.3.0a0",
#     "flags": {},
#     "shapes" : labelme_annotations,
#     "imagePath": "BKF-BLUE-1_phone_%20(55).jpg",
#     "imageData" : None
# }


# with open("./yolo_label/BKF-BLUE-1_phone_%20(55).json", "w") as json_file:
#     json.dump(json_file_full, json_file)

In [33]:
labelme_annotations

[{'label': 'phone',
  'points': [[891, 644], [973, 734]],
  'description': '',
  'shape_type': 'rectangle'},
 {'label': 'person',
  'points': [[797, 666], [1183, 1131]],
  'description': '',
  'shape_type': 'rectangle'}]

# convert yolo to coco class

In [13]:
import os
import json
import cv2
from tqdm import tqdm

In [25]:
# # Define the file path
# file_path = '../assets/crop-img/073/labels/0730_14.txt'

# # Read the file
# with open(file_path, 'r') as file:
#     lines = file.readlines()
#     print(f"old : {lines}")

# # Process the lines
# new_lines = []
# for line in lines:
#     if line.startswith('1 '):
#         new_line = line.replace('1 ', '67 ', 1)
#         new_lines.append(new_line)
#     else:
#         new_lines.append(line)
# print(f"replace to : {new_lines}")

In [30]:
focus_site = "NKA"
input_dir = f'../assets/crop-img/{focus_site}/labels'
output_dir = f'../assets/crop-img/{focus_site}/labels_coco'


label_files = os.listdir(input_dir)
os.makedirs(output_dir, exist_ok=True)
label_files[:5]

['NKA-ch8_1.txt',
 'NKA-ch8_2.txt',
 'NKA-ch8_3.txt',
 'NKA-ch8_4.txt',
 'NKA-ch8_6.txt']

In [31]:
for file_name in tqdm(label_files):
    # Read the input file
    # print(os.path.join(input_dir, file_name))
    # print(input_dir + '/' + file_name)
    with open(os.path.join(input_dir, file_name), 'r') as file:
        lines = file.readlines()
        # print(lines)

    # Process the lines
    new_lines = []
    for line in lines:
        if line.startswith('1 '):
            new_line = line.replace('1 ', '67 ', 1) # change phone to cell phone coco
            new_lines.append(new_line)
        else:
            new_lines.append(line)

    # Write the updated content to the new file
    with open(os.path.join(output_dir,file_name), 'w') as file:
        file.writelines(new_lines)

100%|██████████| 33/33 [00:00<00:00, 552.32it/s]


# Yolo to data studio

### obj. Det

In [34]:
import os
import json
from PIL import Image

In [35]:
# Directory containing images and annotations
image_dir = "../assets/ex_object_det/images"
annotation_dir = "../assets/ex_object_det/labels"

# Supported image extensions
image_extensions = ['.jpg', '.png']
label_map = {0: "dog", 1: "cat"}

In [36]:
# List to store all the converted data
label_studio_data_list = []

In [37]:
# Loop through each annotation file
for annotation_file in os.listdir(annotation_dir):
    if annotation_file.endswith(".txt"):
        # Initialize image_path as None to find the corresponding image
        image_path = None
        
        # Try to find the corresponding image file with different extensions
        for ext in image_extensions:
            image_filename = annotation_file.replace(".txt", ext)
            potential_image_path = os.path.join(image_dir, image_filename)
            if os.path.exists(potential_image_path):
                image_path = potential_image_path
                break
        
        # If no corresponding image was found, skip this annotation file
        if not image_path:
            print(f"No corresponding image found for {annotation_file}, skipping...")
            continue
        
        # Open the image to get its dimensions
        img = Image.open(image_path)
        image_width, image_height = img.size
        print(image_filename)
        
        # Initialize a list to store annotations for this image
        annotations = []
        results = []
        
        # Read YOLO annotations from the file
        with open(os.path.join(annotation_dir, annotation_file), "r") as file:
            i = 0
            for line in file:
                i+=1
                parts = line.strip().split()
                label = int(parts[0])
                x_center = float(parts[1])
                y_center = float(parts[2])
                bbox_width = float(parts[3])
                bbox_height = float(parts[4])
                
                # Convert YOLO format (x_center, y_center, width, height) to Label Studio format (x_min, y_min, width, height)
                x_min = int((x_center - bbox_width / 2) * image_width)
                y_min = int((y_center - bbox_height / 2) * image_height)
                width = int(bbox_width * image_width)
                height = int(bbox_height * image_height)
                
                # Create an annotation entry for Label Studio
                # annotation = {
                #     "label": [str(label)],  # Label Studio requires labels as a list of strings
                #     "coordinates": {
                #         "x": x_min,
                #         "y": y_min,
                #         "width": width,
                #         "height": height
                #     }
                # }
                # annotations.append(annotation)
        
                # Create the final JSON structure for this image
                # ls_annotation = yolo_to_ls(image_path, yolo_labels, image_width, image_height, label_map)
                result = {
                    "id": f"result{i+1}",
                    "type": "rectanglelabels",
                    "from_name": "label",
                    "to_name": "image",
                    "original_width": image_width,
                    "original_height": image_height,
                    "image_rotation": 0,
                    "value": {
                        "rotation": 0,
                        "x": x_min,
                        "y": y_min,
                        "width": width,
                        "height": height,
                        "rectanglelabels": [label_map[label]]
                    }
                }
                results.append(result)
    
        annotation = {
            "data": {
                "image": image_filename
            },
            "predictions": [{
                "model_version": "one",
                "score": 0.5,
                "result": results
            }]
        }

        label_studio_data_list.append(annotation)

# Save all the annotations to a JSON file
with open("label_studio_preannotations4.json", "w") as outfile:
    json.dump(label_studio_data_list, outfile, indent=4)

print("Conversion complete for all images!")

none (1).png
{'data': {'image': 'none (1).png'}, 'predictions': [{'model_version': 'one', 'score': 0.5, 'result': []}]}
none (3).png
{'data': {'image': 'none (3).png'}, 'predictions': [{'model_version': 'one', 'score': 0.5, 'result': []}]}
none (2).png
{'data': {'image': 'none (2).png'}, 'predictions': [{'model_version': 'one', 'score': 0.5, 'result': []}]}
x (1).jpg
{'data': {'image': 'x (1).jpg'}, 'predictions': [{'model_version': 'one', 'score': 0.5, 'result': [{'id': 'result2', 'type': 'rectanglelabels', 'from_name': 'label', 'to_name': 'image', 'original_width': 2546, 'original_height': 1180, 'image_rotation': 0, 'value': {'rotation': 0, 'x': 67, 'y': 155, 'width': 702, 'height': 990, 'rectanglelabels': ['dog']}}, {'id': 'result3', 'type': 'rectanglelabels', 'from_name': 'label', 'to_name': 'image', 'original_width': 2546, 'original_height': 1180, 'image_rotation': 0, 'value': {'rotation': 0, 'x': 982, 'y': 83, 'width': 581, 'height': 1063, 'rectanglelabels': ['dog']}}, {'id': 're

In [23]:
# def yolo_to_ls(image_path, yolo_labels, image_width, image_height, label_map):
#     results = []
    
#     for i, label in enumerate(yolo_labels):
#         class_id, center_x, center_y, width, height = label
#         x = (center_x - width / 2) * 100
#         y = (center_y - height / 2) * 100
#         width = width * 100
#         height = height * 100
        
#         result = {
#             "id": f"result{i+1}",
#             "type": "rectanglelabels",
#             "from_name": "label",
#             "to_name": "image",
#             "original_width": image_width,
#             "original_height": image_height,
#             "image_rotation": 0,
#             "value": {
#                 "rotation": 0,
#                 "x": x,
#                 "y": y,
#                 "width": width,
#                 "height": height,
#                 "rectanglelabels": [label_map[class_id]]
#             }
#         }
#         results.append(result)
    
#     annotation = {
#         "data": {
#             "image": image_path
#         },
#         "predictions": [{
#             "model_version": "one",
#             "score": 0.5,
#             "result": results
#         }]
#     }
    
#     return json.dumps([annotation], indent=2)


In [24]:
# # Example usage
# # Directory containing images and annotations
# image_dir = "../assets/ex_object_det/images"
# annotation_dir = "../assets/ex_object_det/labels"

# # Supported image extensions
# image_extensions = ['.jpg', '.png']


# # Loop through each annotation file
# for annotation_file in os.listdir(annotation_dir):
#     if annotation_file.endswith(".txt"):
#         # Initialize image_path as None to find the corresponding image
#         image_path = None
        
#         # Try to find the corresponding image file with different extensions
#         for ext in image_extensions:
#             image_filename = annotation_file.replace(".txt", ext)
#             potential_image_path = os.path.join(image_dir, image_filename)
#             if os.path.exists(potential_image_path):
#                 image_path = potential_image_path
#                 break
        
#         # If no corresponding image was found, skip this annotation file
#         if not image_path:
#             print(f"No corresponding image found for {annotation_file}, skipping...")
#             continue
        
#         # Open the image to get its dimensions
#         img = Image.open(image_path)
#         image_width, image_height = img.size

# # yolo_labels = [
# #     (0, 0.1, 0.2, 0.3, 0.4),  # Example label: class_id, center_x, center_y, width, height
# #     (1, 0.7, 0.8, 0.05, 0.07)
# # ]
# # image_width = 600
# # image_height = 403
# # label_map = {0: "Airplane", 1: "Car"}  # Mapping from YOLO class IDs to labels

# # ls_annotation = yolo_to_ls(image_path, yolo_labels, image_width, image_height, label_map)
# # print(ls_annotation)

# Other - rewrite label

In [2]:
import os
focus_site = "images"
focus_folder = f"../../../datasets/ID_card/assets/only_license 01-06_2024/datasets onlylicense 01-06_2024/giftt/bad/{focus_site}/"
file_list = os.listdir(focus_folder)
json_files = [filename for filename in file_list if filename.endswith(".json")]
json_files

['driver-IMG_URIKI_LG0_URIKI900153373_LicenseImage_20240412013141.json',
 'driver-IMG_CPFGS_LG0_CPFGS800122499_LicenseImage_20240624050638.json',
 'driver-IMG_CPFGS_LG0_CPFGS800122499_LicenseImage_20240624052252.json',
 'driver-IMG_CPFGS_LG0_CPFGS800122499_LicenseImage_20240624073343.json',
 'driver-IMG_CPFGS_LG0_CPFGS800124939_LicenseImage_20240616195930.json',
 'driver-IMG_CPFGS_LG0_CPFGS800127042_LicenseImage_20240615085655.json',
 'driver-IMG_CPFGS_LG0_CPFGS800146870_LicenseImage_20240216132947.json',
 'driver-IMG_CPFGS_LG0_CPFGS800154190_LicenseImage_20240609134942.json',
 'driver-IMG_CPFGS_LG0_CPFGS800163375_LicenseImage_20240510172128.json',
 'driver-IMG_CPFGS_LG0_CPFGS800163781_LicenseImage_20240205195424.json',
 'driver-IMG_CPFGS_LG0_CPFGS800166569_LicenseImage_20240518182953.json',
 'driver-IMG_CPFGS_LG0_CPFGS800180033_LicenseImage_20240223033537.json',
 'driver-IMG_CPFGS_LG0_CPFGS800182370_LicenseImage_20240501053602.json',
 'driver-IMG_CPFGS_LG0_CPFGS800182370_LicenseImage_

In [3]:
import json

def update_labelme_file(input_file, output_file):
    # Read the JSON file
    with open(input_file, 'r') as file:
        data = json.load(file)
    
    # Update the labels
    for shape in data.get("shapes", []):
        if shape.get("label") == "good":
            shape["label"] = "bad"
    
    # Write the updated data to a new file
    with open(output_file, 'w') as file:
        json.dump(data, file, indent=4)
    
    print(f"Updated file saved to: {output_file}")

for file_name in json_files:
    # Input and output file paths
    input_file = os.path.join(focus_folder,file_name)
    output_file = os.path.join(focus_folder,file_name)

    # Update the label
    update_labelme_file(input_file, output_file)


Updated file saved to: ../../../datasets/ID_card/assets/only_license 01-06_2024/datasets onlylicense 01-06_2024/giftt/bad/images/driver-IMG_URIKI_LG0_URIKI900153373_LicenseImage_20240412013141.json
Updated file saved to: ../../../datasets/ID_card/assets/only_license 01-06_2024/datasets onlylicense 01-06_2024/giftt/bad/images/driver-IMG_CPFGS_LG0_CPFGS800122499_LicenseImage_20240624050638.json
Updated file saved to: ../../../datasets/ID_card/assets/only_license 01-06_2024/datasets onlylicense 01-06_2024/giftt/bad/images/driver-IMG_CPFGS_LG0_CPFGS800122499_LicenseImage_20240624052252.json
Updated file saved to: ../../../datasets/ID_card/assets/only_license 01-06_2024/datasets onlylicense 01-06_2024/giftt/bad/images/driver-IMG_CPFGS_LG0_CPFGS800122499_LicenseImage_20240624073343.json
Updated file saved to: ../../../datasets/ID_card/assets/only_license 01-06_2024/datasets onlylicense 01-06_2024/giftt/bad/images/driver-IMG_CPFGS_LG0_CPFGS800124939_LicenseImage_20240616195930.json
Updated fi